In [1]:
import random
import pandas as pd
import numpy as np
import web3

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation
from schedule import RandomActivationByBreed

In [176]:
def get_battery_lives(model):
    agent_battery_lives = [a.battery_life for a in model.schedule.agents if a.battery_life]
    print(agent_battery_lives)

In [2]:
class Costs():
    
    def __init__(self, record_cost, compute_cost, sign_cost, transmit_cost, stochastic):
        
        self.stochastic = stochastic
        self.record_cost = record_cost
        self.compute_cost = compute_cost
        self.sign_cost = sign_cost
        self.transmit_cost = transmit_cost
        
    def record(self):
        if self.stochastic:
            return self.record_cost # randomness!
        else:
            return self.record_cost
    
    def compute(self):
        if self.stochastic:
            return self.compute_cost # randomness!
        else:
            return self.compute_cost
    
    def sign(self):
        return self.sign_cost
    
    def transmit(self):
        if self.stochastic:
            return self.transmit_cost # randomness!
        else:
            return self.transmit_cost

In [118]:

class Sensor(Agent):
    """
    An empirical sensor with battery, data store,
    processor and information transceiver.

    Attributes:
        unique_id: Universally unique identifier. Here, a randomly generated id.
            This could be a wallet address or public key, plus private key
        battery_life: The amount of battery life. Portrayed in seconds of compute time.
            If -999, the sensor is considered to be connected to a stable power source.
        costs: The energy costs of each action the sensor is capable of taking:
            record(), compute(), sign(), verify()?, and transmit().
        chained: Boolean - whether sensor writes to blockchain or not. Indicates need to 
            invoke .sign() method before .transmit()
        sync_freq: Defines frequency with which sensor transmits data to cloud or blockchain
            n < 1 is probability of transmitting each tick; n > 1 syncs every n ticks


    """
    

    def __init__(self, unique_id, battery_life, costs, 
                 chained, sync_freq, parent_cloud, model):

        super().__init__( unique_id, model )

        self.agent_type = "Sensor"
        self.unique_id = unique_id
        self.battery_life = battery_life
        self.costs = costs
        self.chained = chained
        if chained:
            self.eth_spent = 0
        self.sync_freq = sync_freq
        self.parent_cloud = parent_cloud

    def record(self):
        self.battery_life = self.battery_life - self.costs.record()

    def transmit(self, num_bytes):
        if self.sync_freq < 1.0 and self.model.random.random() < self.sync_freq:
            if self.chained:
                self.eth_spent = self.eth_spent + \
                self.model.schedule._agents[self.parent_cloud].write_data(num_bytes)
                
            self.battery_life = self.battery_life -  self.costs.transmit()
        else:
            if self.model.schedule.steps % self.sync_freq == 0:
                self.battery_life = self.battery_life -  self.costs.transmit()


    def compute(self):
        self.battery_life = self.battery_life - self.costs.compute()
    
    def sign(self):
        if self.chained:
            self.battery_life = self.battery_life - self.costs.sign()
        else:
            pass
        
    def step(self):
        """

        """
        
        if self.battery_life < 0:
            if self.model.verbose:
                print("ID", self.unique_id, 'is out of battery.')
                
                
#             self.model.kill_agents.append(self)

            self.model.schedule.remove(self)
            self.dead = True
            
        self.record()
        self.compute()
        self.sign()
        self.transmit(1)



In [127]:
class Cloud(Agent):
    """
    A traditional cloud server storing and computing data 
    from sensors and serving data to users. 
    
    hourly_cost: 
    uptime: A float value representing guaranteed uptime in SLA
    """
    
      
    def __init__(self, unique_id, ec2_type, usd_per_hour, 
                 usd_per_byte, sample_computation, model):
    
        super().__init__( unique_id, model )

        self.unique_id = unique_id
        self.ec2_type = ec2_type
        self.usd_per_hour = usd_per_hour
        self.usd_per_byte = usd_per_byte
        self.sample_computation = sample_computation

    def write_data(self, num_bytes):
        return 0
        pass
    
    def sample_computation(self):
        pass
    
    def calc_total_cost(self):
        return self.usd_per_hour * self.model.schedule.steps
        
        
    def step(self):
        pass
        


In [100]:
class Blockchain(Agent):
    """
    """
  
    def __init__(self, eth_price, gas_cost, eth_per_byte, eth_per_second):
        
        self.eth_price = eth_price
        self.gas_cost = gas_cost
        self.eth_per_byte = eth_per_byte
        self.eth_per_second = eth_per_second
        
    def write_data(self, num_bytes):
        print(self.eth_per_byte * num_bytes, type(self.eth_per_byte * num_bytes))
        return self.eth_per_byte * num_bytes
        
    def compute_cost_eth(self, num_seconds):
        return self.eth_per_second * num_seconds

    def read_data(self):
        return 0

    def step(self):
        pass

In [166]:
def get_avg_eth_spent(model):
    print(model)
    sensor_eth_spent = [a.eth_spent for a in model.schedule.agents]
    return np.sum(sensor_eth_spent) / len(sensor_eth_spent)

In [189]:
class SensorBlockchainNetwork(Model):
    
    
    def __init__(self, num_sensors, 
                 record_cost, 
                 compute_cost, sign_cost, 
                 transmit_cost, stochastic, 
                 verbose):
        
        super().__init__()
        
        self.verbose = verbose
        
        self.kill_agents = []
        
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            model_reporters = {
                "functional_sensors": lambda m: len(m.schedule.agents)
            },
            
            # Need to work out breed specific report ...
#             agent_reporters = {"Ether": get_avg_eth_spent}
        )
        
        # Now, instantiate each agent and add to schedule
        
        if server_type == "cloud":
            
            self.server = Cloud(self.next_id(), 't2.micro', 0.0059, 0.04, 0.023 / 1000000000, self)
                 # eth_per_byte and AWS costs from Ryan 2017, Example Gas Costs
            
        elif server_type == "blockchain":
            
            self.server = Blockchain(100, 50, 0.006 / 32)
            
        else:
            raise ValueError("Please indicate either 'cloud', or 'blockchain' as cloud_type.")
        

        for i in range(num_sensors):
            costs = Costs(record_cost, compute_cost, sign_cost, 
                          transmit_cost, stochastic)
            
            sensor = Sensor(self.next_id(), 10, costs, 
                            True if i % 2 == 0 else False, 
                            0.1, self.server, self)
            
            self.schedule.add(sensor)
            
  
            
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        
        # Working out a bug described here https://github.com/projectmesa/mesa/issues/352
        for a in self.kill_agents:
            self.schedule.remove(a)
        self.kill_agents = []

    
    @staticmethod
    def count_active_sensors(model):
        count = 0
#         print()
        for agent in model.schedule.agents_by_breed[Sensor]:
            print(agent)
            if not model.schedule.agents_by_breed[Sensor][agent].dead:
                count += 1
        return count
        

In [190]:

m = SensorNetwork(20, 'cloud', 1,1,1,1, False, False)

In [191]:
m.schedule._agents[2].parent_cloud

In [192]:
get_battery_lives(m)

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [193]:
for i in range(0, 100):
    m.step()

KeyError: <__main__.Cloud object at 0x10939b2b0>

In [152]:
agent_d = m.datacollector.get_model_vars_dataframe()

In [153]:
agent_d

,functional_sensors
0,20
1,20
2,20
3,19
4,9
5,0
6,0
7,0
8,0
9,0
